In [ ]:
%%bash
python3 -m pip install kafka-python

In [ ]:
import json
from kafka import KafkaProducer
from kafka.errors import KafkaError

producer = KafkaProducer(bootstrap_servers=['10.10.139.63:9092'],
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

def send_record(rdd):
    records = rdd.collect()
    for r in records:
        producer.send('output', r)
        producer.flush()

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3 pyspark-shell'

from pyspark import SparkContext
import json

sc = SparkContext()
sc.setLogLevel("WARN")

from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils


def country_to_upper(data):
    data['visitor_country'] = data['visitor_country'].upper()
    return data

ssc = StreamingContext(sc, 30)

directKafkaStream = KafkaUtils.createDirectStream(ssc, ['test'], {"metadata.broker.list": '10.10.139.63:9092'}, 
                                                 messageHandler=lambda x: json.loads(x.message))
# Remove message without a 'ts_ingest' field or empty timestamps
filtered_ts = directKafkaStream.filter(lambda x: 'ts_ingest' in x.keys() and x['ts_ingest'])
# Ensure data uniformity, all country codes should be uppercase
uppercase_country = filtered_ts.map(country_to_upper)
# Filter on only clicks from BE
filtered_be = uppercase_country.filter(lambda x: 'visitor_country' in x.keys() and x['visitor_country'] == 'BE')
# Send the result back to Kafka
filtered_be.foreachRDD(send_record)

In [ ]:
ssc.start()
ssc.awaitTermination()